# The Simplex Algorithm

In [ ]:
def simplex_algorithm(A, b, c):
    """
    Solves the linear programming problem using the Simplex algorithm.

    Args:
        A (list of list of float): The matrix representing the coefficients of the constraints.
        b (list of float): The vector representing the right-hand side of the constraints.
        c (list of float): The vector representing the coefficients of the objective function.

    Returns:
        list of list of float: The final tableau after optimization.
    """
    # Initialize tableau
    tableau = initialize_tableau(A, b, c)
    while any(coef < 0 for coef in tableau[0]):
        # Select entering variable
        entering_variable = select_entering_variable(tableau[0])
        # Select leaving variable
        leaving_variable = select_leaving_variable(tableau, entering_variable)
        # Update tableau
        tableau = update_tableau(tableau, entering_variable, leaving_variable)
    return tableau

def initialize_tableau(A, b, c):
    """
    Initializes the tableau for the Simplex algorithm.

    Args:
        A (list of list of float): The matrix representing the coefficients of the constraints.
        b (list of float): The vector representing the right-hand side of the constraints.
        c (list of float): The vector representing the coefficients of the objective function.

    Returns:
        list of list of float: The initialized tableau.
    """
    # Create an identity matrix for the slack variables
    slack_vars = [[0] * len(A) for _ in range(len(A))]
    for i in range(len(A)):
        slack_vars[i][i] = 1

    # Combine A and slack_vars to form the tableau
    tableau = [row_a + row_s + [bi] for row_a, row_s, bi in zip(A, slack_vars, b)]
    # Append the objective function row
    tableau.append([-ci for ci in c] + [0] * (len(A) + 1))

    return tableau

def select_entering_variable(objective_row):
    """
    Selects the entering variable using the most negative coefficient in the objective row.

    Args:
        objective_row (list of float): The objective row of the tableau.

    Returns:
        int: The index of the entering variable.
    """
    min_value = min(objective_row)
    return objective_row.index(min_value)

def select_leaving_variable(tableau, entering_variable):
    """
    Selects the leaving variable using the minimum ratio test.

    Args:
        tableau (list of list of float): The current tableau.
        entering_variable (int): The index of the entering variable.

    Returns:
        int: The index of the leaving variable.
    """
    ratios = []
    for row in tableau[:-1]:
        if row[entering_variable] > 0:
            ratios.append(row[-1] / row[entering_variable])
        else:
            ratios.append(float('inf'))
    return ratios.index(min(ratios))

def update_tableau(tableau, entering_variable, leaving_variable):
    """
    Updates the tableau for the Simplex algorithm.

    Args:
        tableau (list of list of float): The current tableau.
        entering_variable (int): The index of the entering variable.
        leaving_variable (int): The index of the leaving variable.

    Returns:
        list of list of float: The updated tableau.
    """
    new_tableau = [row[:] for row in tableau]
    pivot = tableau[leaving_variable][entering_variable]

    # Divide the leaving row by the pivot element
    new_tableau[leaving_variable] = [x / pivot for x in tableau[leaving_variable]]

    # Update the rest of the tableau
    for i in range(len(tableau)):
        if i != leaving_variable:
            ratio = tableau[i][entering_variable] / pivot
            new_tableau[i] = [x - ratio * y for x, y in zip(tableau[i], new_tableau[leaving_variable])]

    return new_tableau

# Example usage:
A = [[2, 1], [1, 2]]
b = [20, 20]
c = [10, 12]
final_tableau = simplex_algorithm(A, b, c)
print("Final Tableau:")
for row in final_tableau:
    print(row)

# Ellipsoid Algorithm

In [ ]:
import numpy as np

def ellipsoid(A, b, c, epsilon):
    """
    Solves the linear programming problem using the Ellipsoid method.

    Args:
        A (numpy.ndarray): The matrix representing the coefficients of the constraints.
        b (numpy.ndarray): The vector representing the right-hand side of the constraints.
        c (numpy.ndarray): The vector representing the coefficients of the objective function.
        epsilon (float): The desired accuracy for the solution.

    Returns:
        numpy.ndarray: The optimal solution.
    """
    m, n = A.shape
    # Initialize ellipsoid containing the feasible region
    E = Ellipsoid(np.zeros(n), np.eye(n))
    while E.volume() > epsilon:
        # Compute the center of the ellipsoid
        x_k = E.center()
        # Check if the current center is feasible
        if np.all(np.dot(A, x_k) <= b):
            return x_k  # Return optimal solution
        else:
            # Compute a separating hyperplane between x_k and the feasible region
            hyperplane_normal = np.dot(A.T, np.linalg.solve(np.dot(A, np.dot(E.C, A.T)), np.dot(A, x_k) - b))
            # Update the ellipsoid based on the separating hyperplane
            E.update(hyperplane_normal)
    return E.center()

class Ellipsoid:
    def __init__(self, center, C):
        """
        Initializes an ellipsoid with a given center and shape matrix.

        Args:
            center (numpy.ndarray): The center of the ellipsoid.
            C (numpy.ndarray): The shape matrix of the ellipsoid.
        """
        self.center = center
        self.C = C

    def volume(self):
        """
        Calculates the volume of the ellipsoid.

        Returns:
            float: The volume of the ellipsoid.
        """
        return np.pi ** (self.C.shape[0] / 2) / np.sqrt(np.linalg.det(self.C))

    def update(self, normal):
        """
        Updates the ellipsoid based on a separating hyperplane.

        Args:
            normal (numpy.ndarray): The normal vector of the separating hyperplane.
        """
        self.center += np.dot(np.linalg.inv(self.C), normal / np.linalg.norm(normal))
        self.C = self.C / np.linalg.norm(normal) ** 2

    def center(self):
        """
        Returns the center of the ellipsoid.

        Returns:
            numpy.ndarray: The center of the ellipsoid.
        """
        return self.center

# Example usage
A = np.array([[2, 1], [1, 2]])
b = np.array([4, 5])
c = np.array([-3, -5])
epsilon = 1e-6
optimal_solution = ellipsoid(A, b, c, epsilon)
print("Optimal solution:", optimal_solution)

# Revised Simplex Method

In [ ]:
import numpy as np

def revised_simplex(A, b, c):
    """
    Solves the linear programming problem using the revised simplex method.

    Args:
        A (numpy.ndarray): The matrix representing the coefficients of the constraints.
        b (numpy.ndarray): The vector representing the right-hand side of the constraints.
        c (numpy.ndarray): The vector representing the coefficients of the objective function.

    Returns:
        tuple: The objective value and the optimal solution.
    """
    m, n = A.shape
    # Initialize basic feasible solution
    B = np.eye(m)
    N = np.eye(n)
    x_B = np.zeros(m)
    x_N = np.linalg.solve(B, b)
    while np.any(c @ N - np.dot(x_N, A) < 0):
        # Choose entering variable with negative reduced cost
        enter_idx = np.argmax(c @ N - np.dot(x_N, A) < 0)
        # Determine leaving variable using ratio test
        ratios = np.divide(x_N, np.dot(B[:, enter_idx], A[:, enter_idx]))
        leave_idx = np.argmin(ratios[ratios > 0])
        leave_idx = np.where(ratios > 0)[leave_idx]
        # Update basic solution
        pivot_row = B[leave_idx, :]
        pivot_col = A[:, enter_idx]
        x_B[leave_idx] = b[leave_idx] / pivot_row @ pivot_col
        B[leave_idx, :] = pivot_col / pivot_row @ B
        N[enter_idx, :] = -pivot_col / pivot_row @ N
        N[enter_idx, enter_idx] = 1 / pivot_row @ N
        x_N = np.linalg.solve(B, b)
    return c @ x_N, x_N

# Example usage
A = np.array([[2, 1], [1, 2], [1, -1]])
b = np.array([4, 5, 1])
c = np.array([-3, -5])
objective_value, optimal_solution = revised_simplex(A, b, c)
print("Objective value:", objective_value)
print("Optimal solution:", optimal_solution)

# Newton's Method

In [ ]:
import numpy as np

def newton_method(f, f_prime, x0, tol=1e-7, max_iter=100):
    """
    Solves f(x) = 0 using Newton's method.

    Args:
        f (function): The function for which we want to find a root.
        f_prime (function): The derivative of the function f.
        x0 (float): Initial guess for the root.
        tol (float): Tolerance for the convergence criterion. Defaults to 1e-7.
        max_iter (int): Maximum number of iterations. Defaults to 100.

    Returns:
        float: The estimated root.
        int: The number of iterations performed.
    """
    x = x0
    for i in range(max_iter):
        fx = f(x)
        fpx = f_prime(x)
        if fpx == 0:
            raise ValueError("Derivative is zero. No solution found.")

        # Compute the next approximation
        x_new = x - fx / fpx

        # Check for convergence
        if abs(x_new - x) < tol:
            return x_new, i + 1

        # Update the current approximation
        x = x_new

    raise ValueError("Maximum number of iterations reached. No solution found.")

# Example usage
def f(x):
    return x**2 - 2  # Example function: f(x) = x^2 - 2

def f_prime(x):
    return 2 * x  # Derivative: f'(x) = 2x

x0 = 1.0  # Initial guess
root, iterations = newton_method(f, f_prime, x0)
print("Estimated root:", root)
print("Iterations:", iterations)

# Barrier Method

In [ ]:
import numpy as np
from scipy.optimize import minimize

def barrier_method(f, grad_f, hess_f, constraints, x0, t=1.0, mu=10, tol=1e-7, max_iter=100):
    """
    Solves a constrained optimization problem using the Barrier method.

    Args:
        f (function): The objective function to minimize.
        grad_f (function): The gradient of the objective function.
        hess_f (function): The Hessian of the objective function.
        constraints (list): A list of constraint functions g_i(x) <= 0.
        x0 (numpy.ndarray): Initial guess for the variables.
        t (float): Initial barrier parameter.
        mu (float): Factor by which to increase t.
        tol (float): Tolerance for the convergence criterion.
        max_iter (int): Maximum number of iterations.

    Returns:
        numpy.ndarray: The estimated solution.
    """
    def barrier_function(x, t):
        penalty = 0
        for g in constraints:
            penalty += -np.log(-g(x))
        return t * f(x) + penalty

    def barrier_grad(x, t):
        penalty_grad = np.zeros_like(x)
        for g in constraints:
            penalty_grad += grad_f(x) / -g(x)
        return t * grad_f(x) + penalty_grad

    def barrier_hess(x, t):
        penalty_hess = np.zeros((x.shape[0], x.shape[0]))
        for g in constraints:
            grad_g = grad_f(x) / -g(x)
            penalty_hess += np.outer(grad_g, grad_g) / g(x)
        return t * hess_f(x) + penalty_hess

    x = x0
    for i in range(max_iter):
        result = minimize(barrier_function, x, args=(t), jac=barrier_grad, hess=barrier_hess, method='trust-constr')
        x = result.x
        if len(constraints) / t < tol:
            break
        t *= mu

    return x

# Example usage
def f(x):
    return x[0]**2 + x[1]**2  # Example objective function

def grad_f(x):
    return np.array([2*x[0], 2*x[1]])  # Gradient of the objective function

def hess_f(x):
    return np.array([[2, 0], [0, 2]])  # Hessian of the objective function

def g1(x):
    return x[0] + x[1] - 1  # Example constraint: x[0] + x[1] <= 1

def g2(x):
    return -x[0]  # Example constraint: x[0] >= 0

def g3(x):
    return -x[1]  # Example constraint: x[1] >= 0

constraints = [g1, g2, g3]
x0 = np.array([0.5, 0.5])  # Initial guess

optimal_solution = barrier_method(f, grad_f, hess_f, constraints, x0)
print("Optimal solution:", optimal_solution)

# Path-following Method

In [ ]:
import numpy as np
from scipy.optimize import minimize

def path_following_method(f, grad_f, hess_f, constraints, x0, tol=1e-7, max_iter=100):
    """
    Solves a constrained optimization problem using the Path-following method.

    Args:
        f (function): The objective function to minimize.
        grad_f (function): The gradient of the objective function.
        hess_f (function): The Hessian of the objective function.
        constraints (list): A list of constraint functions g_i(x) <= 0.
        x0 (numpy.ndarray): Initial guess for the variables.
        tol (float): Tolerance for the convergence criterion.
        max_iter (int): Maximum number of iterations.

    Returns:
        numpy.ndarray: The estimated solution.
    """
    def barrier_function(x, t):
        penalty = 0
        for g in constraints:
            penalty += -np.log(-g(x))
        return t * f(x) + penalty

    def barrier_grad(x, t):
        penalty_grad = np.zeros_like(x)
        for g in constraints:
            penalty_grad += grad_f(x) / -g(x)
        return t * grad_f(x) + penalty_grad

    def barrier_hess(x, t):
        penalty_hess = np.zeros((x.shape[0], x.shape[0]))
        for g in constraints:
            grad_g = grad_f(x) / -g(x)
            penalty_hess += np.outer(grad_g, grad_g) / g(x)
        return t * hess_f(x) + penalty_hess

    x = x0
    t = 1.0  # Start with an initial barrier parameter
    mu = 10  # Update factor for the barrier parameter

    for i in range(max_iter):
        result = minimize(barrier_function, x, args=(t), jac=barrier_grad, hess=barrier_hess, method='trust-constr')
        x = result.x
        if np.linalg.norm(result.jac) < tol:
            break
        t *= mu  # Increase the barrier parameter

    return x

# Example usage
def f(x):
    return x[0]**2 + x[1]**2  # Example objective function

def grad_f(x):
    return np.array([2*x[0], 2*x[1]])  # Gradient of the objective function

def hess_f(x):
    return np.array([[2, 0], [0, 2]])  # Hessian of the objective function

def g1(x):
    return x[0] + x[1] - 1  # Example constraint: x[0] + x[1] <= 1

def g2(x):
    return -x[0]  # Example constraint: x[0] >= 0

def g3(x):
    return -x[1]  # Example constraint: x[1] >= 0

constraints = [g1, g2, g3]
x0 = np.array([0.5, 0.5])  # Initial guess

optimal_solution = path_following_method(f, grad_f, hess_f, constraints, x0)
print("Optimal solution:", optimal_solution)

# Gomory's Method

In [ ]:
import numpy as np
from scipy.optimize import linprog

def gomory_method(A, b, c):
    """
    Solves a linear programming problem using Gomory's cutting plane method.

    Args:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        c (numpy.ndarray): Cost vector.

    Returns:
        numpy.ndarray: The optimal solution.
    """
    m, n = A.shape
    bounds = [(0, None) for _ in range(n)]

    while True:
        # Solve the current LP relaxation
        res = linprog(c, A_eq=A, b_eq=b, bounds=bounds, method='simplex')
        x = res.x

        # Check if the solution is integer
        if np.allclose(x, np.round(x)):
            return np.round(x)

        # Find the most fractional variable
        fractional_indices = np.where(np.abs(x - np.round(x)) > 1e-5)[0]
        if len(fractional_indices) == 0:
            return x  # No fractional variables found, return the solution

        # Add a Gomory cut
        i = fractional_indices[0]
        frac_part = x[i] - np.floor(x[i])
        new_row = np.floor(A[i]) - A[i]
        new_b = frac_part - np.floor(x[i])

        A = np.vstack([A, new_row])
        b = np.append(b, new_b)

# Example usage
A = np.array([[1, 1], [2, 1]])
b = np.array([4, 5])
c = np.array([-3, -4])

optimal_solution = gomory_method(A, b, c)
print("Optimal solution:", optimal_solution)

# Chvátal-Gomory Cutting Plane Method

In [ ]:
import numpy as np
from scipy.optimize import linprog

def chvatal_gomory_cutting_plane(A, b, c):
    """
    Solves a linear programming problem using the Chvátal-Gomory cutting plane method.

    Args:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        c (numpy.ndarray): Cost vector.

    Returns:
        numpy.ndarray: The optimal solution.
    """
    m, n = A.shape
    bounds = [(0, None) for _ in range(n)]

    while True:
        # Solve the current LP relaxation
        res = linprog(c, A_eq=A, b_eq=b, bounds=bounds, method='simplex')
        x = res.x

        # Check if the solution is integer
        if np.allclose(x, np.round(x)):
            return np.round(x)

        # Find the most fractional row in the constraint matrix
        fractional_rows = [i for i in range(len(b)) if not np.isclose(b[i], np.round(b[i]))]
        if not fractional_rows:
            return x  # No fractional constraints found, return the solution

        # Add a Chvátal-Gomory cut
        i = fractional_rows[0]
        new_row = np.floor(A[i]) - A[i]
        new_b = np.floor(b[i])

        A = np.vstack([A, new_row])
        b = np.append(b, new_b)

# Example usage
A = np.array([[1, 1], [2, 1]])
b = np.array([4, 5])
c = np.array([-3, -4])

optimal_solution = chvatal_gomory_cutting_plane(A, b, c)
print("Optimal solution:", optimal_solution)

# Lift-and-Project Method

In [ ]:
import numpy as np
from scipy.optimize import linprog

def lift_and_project(A, b, c, max_iterations=100, tol=1e-6):
    """
    Solves a linear programming problem using the Lift-and-Project method.

    Args:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        c (numpy.ndarray): Cost vector.
        max_iterations (int): Maximum number of iterations.
        tol (float): Tolerance for checking integrality.

    Returns:
        numpy.ndarray: The optimal solution.
    """
    m, n = A.shape
    bounds = [(0, None) for _ in range(n)]
    A_ = np.copy(A)
    b_ = np.copy(b)

    for iteration in range(max_iterations):
        # Solve the current LP relaxation
        res = linprog(c, A_eq=A_, b_eq=b_, bounds=bounds, method='simplex')
        x = res.x

        # Check if the solution is integer
        if np.allclose(x, np.round(x), atol=tol):
            return np.round(x)

        # Find the most fractional variable
        fractional_vars = [i for i in range(n) if not np.isclose(x[i], np.round(x[i]), atol=tol)]
        if not fractional_vars:
            return x  # No fractional variables found, return the solution

        # Lift-and-Project: Add new constraints to eliminate the current fractional solution
        i = fractional_vars[0]
        floor_xi = np.floor(x[i])
        ceil_xi = np.ceil(x[i])

        # Create new rows for the lift-and-project cuts
        new_row_1 = np.zeros(n)
        new_row_2 = np.zeros(n)

        new_row_1[i] = 1
        new_row_2[i] = -1

        A_ = np.vstack([A_, new_row_1, new_row_2])
        b_ = np.append(b_, floor_xi, -ceil_xi)

    return x

# Example usage
A = np.array([[1, 1], [2, 1]])
b = np.array([4, 5])
c = np.array([-3, -4])

optimal_solution = lift_and_project(A, b, c)
print("Optimal solution:", optimal_solution)

# Stochastic Linear Programming

In [ ]:
import numpy as np
from scipy.optimize import linprog

def sample_scenarios(num_scenarios, A_stochastic, b_stochastic):
    """
    Generate sample scenarios for the stochastic elements of A and b.

    Args:
        num_scenarios (int): Number of scenarios to generate.
        A_stochastic (list of tuples): List of tuples representing the stochastic part of A.
        b_stochastic (list of tuples): List of tuples representing the stochastic part of b.

    Returns:
        list of numpy.ndarray: Sampled scenarios for A.
        list of numpy.ndarray: Sampled scenarios for b.
    """
    A_scenarios = []
    b_scenarios = []

    for _ in range(num_scenarios):
        A_scenario = np.zeros_like(A_stochastic[0][0])
        for (mean, std) in A_stochastic:
            A_scenario += np.random.normal(mean, std)
        A_scenarios.append(A_scenario)

        b_scenario = np.zeros_like(b_stochastic[0][0])
        for (mean, std) in b_stochastic:
            b_scenario += np.random.normal(mean, std)
        b_scenarios.append(b_scenario)

    return A_scenarios, b_scenarios

def stochastic_linear_programming(A_deterministic, b_deterministic, c, A_stochastic, b_stochastic, num_scenarios=100):
    """
    Solve a stochastic linear programming problem using the sample average approximation method.

    Args:
        A_deterministic (numpy.ndarray): Deterministic part of the coefficient matrix A.
        b_deterministic (numpy.ndarray): Deterministic part of the right-hand side vector b.
        c (numpy.ndarray): Cost vector.
        A_stochastic (list of tuples): List of tuples representing the stochastic part of A.
        b_stochastic (list of tuples): List of tuples representing the stochastic part of b.
        num_scenarios (int): Number of scenarios to generate.

    Returns:
        numpy.ndarray: The optimal solution.
    """
    m, n = A_deterministic.shape
    A_scenarios, b_scenarios = sample_scenarios(num_scenarios, A_stochastic, b_stochastic)

    # Initialize the expected value of the constraint matrix and right-hand side vector
    A_expected = np.copy(A_deterministic)
    b_expected = np.copy(b_deterministic)

    for A_scenario, b_scenario in zip(A_scenarios, b_scenarios):
        A_expected += A_scenario
        b_expected += b_scenario

    # Average the scenarios to approximate the expected values
    A_expected /= num_scenarios
    b_expected /= num_scenarios

    # Solve the approximated deterministic linear program
    res = linprog(c, A_eq=A_expected, b_eq=b_expected, method='simplex')
    return res.x

# Example usage
A_deterministic = np.array([[1, 1], [2, 1]])
b_deterministic = np.array([4, 5])
c = np.array([-3, -4])
A_stochastic = [(np.array([[0.1, 0.1], [0.2, 0.2]]), np.array([[0.01, 0.01], [0.02, 0.02]]))]
b_stochastic = [(np.array([0.1, 0.2]), np.array([0.01, 0.02]))]

optimal_solution = stochastic_linear_programming(A_deterministic, b_deterministic, c, A_stochastic, b_stochastic)
print("Optimal solution:", optimal_solution)

# Multi-objective Linear Programming

In [ ]:
import numpy as np
from scipy.optimize import linprog

def weighted_sum_method(A, b, c_list, weights):
    """
    Solve a multi-objective linear programming problem using the weighted sum method.

    Args:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        c_list (list of numpy.ndarray): List of cost vectors for each objective.
        weights (list of float): Weights for each objective.

    Returns:
        numpy.ndarray: The optimal solution.
        float: The optimal weighted objective value.
    """
    # Validate input dimensions
    num_objectives = len(c_list)
    if num_objectives != len(weights):
        raise ValueError("The number of cost vectors and weights must be the same.")

    # Compute the weighted sum of the cost vectors
    c_weighted = np.zeros_like(c_list[0])
    for c, w in zip(c_list, weights):
        c_weighted += w * c

    # Solve the single-objective linear program with the weighted cost vector
    res = linprog(c_weighted, A_eq=A, b_eq=b, method='simplex')
    return res.x, res.fun

# Example usage
A = np.array([[1, 2], [4, 0], [0, 4]])
b = np.array([8, 16, 12])
c_list = [np.array([1, 1]), np.array([2, 3])]
weights = [0.5, 0.5]

optimal_solution, optimal_value = weighted_sum_method(A, b, c_list, weights)
print("Optimal solution:", optimal_solution)
print("Optimal weighted objective value:", optimal_value)

# Vehicle Routing Problem using Linear Programming

In [ ]:
import pulp
import numpy as np

def solve_vrp(num_vehicles, vehicle_capacity, demands, distance_matrix):
    num_customers = len(demands) - 1  # excluding the depot

    # Create the problem variable
    prob = pulp.LpProblem("VRP", pulp.LpMinimize)

    # Decision variables
    x = pulp.LpVariable.dicts("x", [(i, j, k) for i in range(num_customers + 1) for j in range(num_customers + 1) for k in range(num_vehicles)], 0, 1, pulp.LpBinary)
    u = pulp.LpVariable.dicts("u", (i for i in range(1, num_customers + 1)), 0, vehicle_capacity, pulp.LpContinuous)

    # Objective function: minimize the total distance traveled
    prob += pulp.lpSum(distance_matrix[i][j] * x[i, j, k] for i in range(num_customers + 1) for j in range(num_customers + 1) for k in range(num_vehicles))

    # Constraints
    # Each customer is visited exactly once by exactly one vehicle
    for j in range(1, num_customers + 1):
        prob += pulp.lpSum(x[i, j, k] for i in range(num_customers + 1) for k in range(num_vehicles)) == 1

    # Each vehicle starts from the depot
    for k in range(num_vehicles):
        prob += pulp.lpSum(x[0, j, k] for j in range(1, num_customers + 1)) == 1

    # Each vehicle returns to the depot
    for k in range(num_vehicles):
        prob += pulp.lpSum(x[i, 0, k] for i in range(1, num_customers + 1)) == 1

    # Flow constraints
    for k in range(num_vehicles):
        for i in range(num_customers + 1):
            prob += pulp.lpSum(x[i, j, k] for j in range(num_customers + 1)) - pulp.lpSum(x[j, i, k] for j in range(num_customers + 1)) == 0

    # Subtour elimination constraints
    for k in range(num_vehicles):
        for i in range(1, num_customers + 1):
            for j in range(1, num_customers + 1):
                if i != j:
                    prob += u[i] - u[j] + vehicle_capacity * x[i, j, k] <= vehicle_capacity - demands[j]

    # Solve the problem
    prob.solve()

    # Get the results
    routes = []
    for k in range(num_vehicles):
        route = []
        i = 0
        while True:
            for j in range(num_customers + 1):
                if x[i, j, k].varValue > 0.5:
                    route.append(j)
                    i = j
                    break
            if i == 0:
                break
        routes.append(route)

    return routes

# Example usage
num_vehicles = 2
vehicle_capacity = 15
demands = [0, 4, 6, 7, 3, 5]  # Including the depot with demand 0
distance_matrix = [
    [0, 10, 15, 20, 25, 30],
    [10, 0, 35, 25, 30, 40],
    [15, 35, 0, 30, 20, 25],
    [20, 25, 30, 0, 15, 10],
    [25, 30, 20, 15, 0, 35],
    [30, 40, 25, 10, 35, 0]
]

routes = solve_vrp(num_vehicles, vehicle_capacity, demands, distance_matrix)
print("Routes:", routes)


# Financial Portfolio Optimization using Linear Programming

In [ ]:
import pulp
import numpy as np

def portfolio_optimization(expected_returns, cov_matrix, risk_tolerance):
    n_assets = len(expected_returns)

    # Create the problem variable
    prob = pulp.LpProblem("Portfolio Optimization", pulp.LpMaximize)

    # Decision variables for asset weights
    weights = pulp.LpVariable.dicts("weights", range(n_assets), lowBound=0, upBound=1)

    # Objective function: maximize expected return
    prob += pulp.lpSum(expected_returns[i] * weights[i] for i in range(n_assets)), "Expected Return"

    # Constraint: sum of weights equals 1 (fully invested portfolio)
    prob += pulp.lpSum(weights[i] for i in range(n_assets)) == 1, "Total Weight"

    # Constraint: portfolio variance should be less than or equal to risk tolerance
    portfolio_variance = pulp.lpSum(weights[i] * weights[j] * cov_matrix[i][j] for i in range(n_assets) for j in range(n_assets))
    prob += portfolio_variance <= risk_tolerance, "Risk Constraint"

    # Solve the problem
    prob.solve()

    # Get the results
    optimal_weights = np.array([pulp.value(weights[i]) for i in range(n_assets)])
    return optimal_weights

# Example usage
expected_returns = [0.1, 0.2, 0.15]  # Example expected returns of assets
cov_matrix = [[0.005, -0.010, 0.004], [-0.010, 0.040, -0.002], [0.004, -0.002, 0.023]]  # Example covariance matrix
risk_tolerance = 0.005  # Maximum acceptable risk (variance)

optimal_weights = portfolio_optimization(expected_returns, cov_matrix, risk_tolerance)
print("Optimal Weights:", optimal_weights)